In [ ]:
from typing_extensions import Literal
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.types import Command
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_core.messages import convert_to_messages


In [3]:
load_dotenv()

False

In [ ]:
openai_model = ChatOpenAI(model="gpt-4o")

In [ ]:
groq_model = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [ ]:
openai_model.invoke("hi")

In [ ]:
groq_model.invoke("hi")

In [8]:
def add_numbers(state):
    results = state["num1"] + state["num2"]
    print(f"addition result: {results}")
    return Command(goto="multiply", update={"sum": results})

In [9]:
state = {"num1": 1, "num2": 2}

In [10]:
print(add_numbers(state))

addition result: 3
Command(update={'sum': 3}, goto='multiply')


In [4]:
@tool
def transfer_to_multiplication_expert():
    """ Ask multiplication agent for help """
    return

In [5]:
@tool
def transfer_to_additional_expert():
    """ Ask addition agent for help """
    return

In [ ]:
model_with_tool = openai_model.bind_tools([transfer_to_multiplication_expert])

In [ ]:
ai_message = model_with_tool.invoke("hi")

In [ ]:
ai_message.tool_calls

In [ ]:
ai_message = model_with_tool.invoke("What's (3 + 9) * 54. provide me the output")

In [ ]:
ai_message.tool_calls

In [6]:
def additional_expert(state:MessagesState) -> Command[Literal["multiplication_expert", "__end__"]]:
    system_prompt = (
        "You are an addition expert. you can ask the multiplication expert for help with multiplication."
        "Always do your portion of calculation before the handoff."
    )

    messages = [{"role": "system", "content": system_prompt()}] + state["messages"]
    ai_msg = openai_model.bind_tools([transfer_to_multiplication_expert]).invoke(messages)
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id
        }

        return Command(
            goto="multiplitcational_expert", update={"messages": [ai_msg, tool_msg]}
        )
    
    return {"messages": [ai_msg]}

In [7]:
def multiplication_expert(state:MessagesState) -> Command[Literal["additional_expert", "__end__"]]:
    system_prompt = (
        "You are an multiplication expert. you can ask the addition expert for help with addition."
        "Always do your portion of calculation before the handoff."
    )

    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    ai_msg = openai_model.bind_tools([transfer_to_additional_expert]).invoke(messages)

    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id
        }

        return Command(goto="additional_expert", update={"messages": [ai_msg, tool_msg]})
    
    return {"messages": [ai_msg]}

In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("additional_expert", additional_expert)
workflow.add_node("multiplication_expert", multiplication_expert)

workflow.add_edge(START, "additional_expert")

app = workflow.compile()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
app.invoke({"messages": "What's (3 + 3) * 53. provide me the output"})

In [ ]:
def pretty_print_messages(updata):
    if isinstance(updata, tuple):
        ns, updata = updata
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return
        
        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}")
        print("\n")

    for node_name, node_update in updata.items():
        print(f"Update from node {node_name}")
        print("\n")

        for m in convert_to_messages(node_update["messages"]):
            m.pretty_print()
        print("\n")

In [ ]:
# Let's run the graph with an expression that requires both additional and multiplication:
for chunk in app.stream(
    {"messages": [("user", "What's (3 + 6) * 53. provide me the output")]}
):
    print("**** Chunk....****")

    pretty_print_messages(chunk)

In [ ]:
@tool
def transfer_to_travel_advisor():
    """ Ask travel advisor for help. """
    return

@tool
def transfer_to_hotel_advisor():
    """ Ask hotel advisor for help. """
    return

In [ ]:
def travel_advisor(state: MessagesState) -> Command[Literal["hotel_advisor", "__end__"]]:
    system_prompr = (
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc)."
        "If you need hotel recommendations, ask 'hotel_advisor' for help."
    )

    messages = [{"role": "system", "content": system_prompr}] + state["messages"]

    ai_msg = openai_model.bind_tools([transfer_to_hotel_advisor]).invoke(messages)

    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id
        }

        return Command(goto="hotel_advisor", update={"messages": [ai_msg, tool_msg]})
    
    return {"messages": [ai_msg]}

In [ ]:
def hotel_advisor(state: MessagesState) -> Command[Literal["travel_advisor", "__end__"]]:
    system_prompr = (
        "You are a hotel expert that can provide hotel recommendations for a given distination."
        "If you need help picking travel destinations, ask 'travel_advisor' for help."
    )

    messages = [{"role": "system", "content": system_prompr}] + state["messages"]

    ai_msg = openai_model.bind_tools([transfer_to_travel_advisor]).invoke(messages)

    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id
        }

        return Command(goto="travel_advisor", update={"messages": [ai_msg, tool_msg]})
    
    return {"messages": [ai_msg]}

In [ ]:
workflow_1 = StateGraph(MessagesState)

workflow_1.add_node("travel_advisor", travel_advisor)
workflow_1.add_node("hotel_advisor", hotel_advisor)

workflow_1.add_edge(START, "travel_advisor")

app_1 = workflow_1.compile()

In [ ]:
display(Image(app_1.get_graph().draw_mermaid_png()))

In [ ]:
input_1 = ({"messages": [("user", "I am planning a trip to California in the USA from Tehran. What are the best flight routes and Can you guide me on suggesting the best hotel?")]})

In [ ]:
app_1.invoke(input_1)

In [ ]:
for chunk in app_1.stream(
    input_1
):
    print("**** Chunk....****")

    pretty_print_messages(chunk)